In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
df = pd.read_csv("./data/fake_news_data.csv")

In [ ]:
df.head()

In [ ]:
unique_domains = list(df["source_domain"].unique())
unique_titles = list(df["title"].unique())

In [ ]:
num_words=10000
num_domains = len(unique_domains)

In [ ]:

title_input = keras.Input(shape=(1,), name="title", dtype=tf.string)
source_domain_input = keras.Input(shape=(1,), name="domain", dtype=tf.string)

tweet_num_input = keras.Input(shape=(1,), dtype="int32", name="tweet_num")

title_vectorizer = keras.layers.TextVectorization(max_tokens=num_words)
title_vectorizer.adapt(unique_titles)

title_vector = title_vectorizer(title_input)

source_domain_lookup = keras.layers.StringLookup(vocabulary=unique_domains)(source_domain_input)

In [ ]:
title_features=layers.Embedding(num_words+1, 64)(title_vector)
source_domain_embedding = layers.Embedding(num_domains+1, 64)(source_domain_lookup)
tweet_num_normalized = layers.Normalization()
tweet_num_normalized.adapt(df["tweet_num"])

In [ ]:
title_features = layers.LSTM(128)(title_features)
source_domain_features = layers.GlobalAveragePooling1D()(source_domain_embedding)

In [ ]:
x = layers.concatenate([title_features, source_domain_features])

In [ ]:
is_real_pred = layers.Dense(1, name="real")(x)

In [ ]:
model = keras.Model(inputs=[title_input, source_domain_input], outputs=[is_real_pred])

In [ ]:
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=["binary_accuracy"]
)

In [ ]:
import numpy as np

In [ ]:
title_data = np.array((df["title"].astype(str)))
domain_data = np.array(df["source_domain"].astype(str))

target = np.array(df["real"].astype(int))

In [ ]:
history = model.fit({"title":title_data, 
            "domain":domain_data},
            {"real":target}, 
            epochs=100)